In [ ]:
import pandas as pd
from scipy.interpolate import interp1d
from scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import price_data as price
import statistics
import time
from datetime import date
import os
from gql import gql, Client
from gql.transport.aiohttp import AIOHTTPTransport

# Select your transport with a defined url endpoint
transport = AIOHTTPTransport(url="https://saturn.hasura.app/v1/graphql", headers={'x-hasura-admin-secret': 'Rc07SJt4ryC6RyNXDKFRAtFmRkGBbT8Ez3SdaEYsHQoHemCldvs52Kc803oK8X62'})

# Create a GraphQL client using the defined transport
client = Client(transport=transport, fetch_schema_from_transport=True)

async def get_stored_data(symbol,timeframe):
    # Provide a GraphQL query
    split_symbol=symbol.split('/')
    base_currency=split_symbol[0]
    quote_currency=split_symbol[1]
    table=base_currency+quote_currency+'_'+timeframe
    if table=='BTCUSD_1d':
        query = gql(
            """
            query MyQuery {
                BTCUSD_1d {
                    unix
                    close
                    high
                    low
                    open
                }
            }
        """
        )
    elif table=='ETHUSD_1d':
        query = gql(
            """
            query MyQuery {
                ETHUSD_1d {
                    unix
                    close
                    high
                    low
                    open
                }
            }
        """
        )
    elif table=='ETHBTC_1d':
        query = gql(
            """
            query MyQuery {
                ETHBTC_1d {
                    unix
                    close
                    high
                    low
                    open
                }
            }
        """
        )
    else:
        return 'no such table'

    # Execute the query on the transport
    result = await client.execute_async(query)
    candles=result[table]
    df=pd.DataFrame({},columns=['unix','close','high','low','open'])
    for candle in candles:
        df=df.append(candle,ignore_index=True)

    return df.sort_values(by=['unix'], ignore_index=True)

    return result

def find_start(timestamps):
    start_found=False
    timestamps=list(map(lambda x:x[0]/1000,timestamps))
    index=len(timestamps)-1
    while not(start_found):
        print(timestamps[index])
        day=date.fromtimestamp(timestamps[index]).weekday()
        if day==0:
            start_found=True
        else:   
            index=index-1
    return index

def read_data(filename):
    raw=pd.read_csv('data/'+filename)
    timestamps=raw['unix']
    for i in range(len(timestamps)):
        if np.log10(timestamps.iloc[i])<12:
            raw.loc[i,'unix']=timestamps[i]*1000
    return raw.sort_values(by=['unix'], ignore_index=True)

btc_price_data= await get_stored_data('BTC/USD','1d')
btc_price_data

In [ ]:
weekly_candles=price.get_price_data('1w',data=btc_price_data)
weekly_candles

In [ ]:
def update_csv(symbol,timeframe):
    
    split_symbol=symbol.split('/')
    base_currency=split_symbol[0]
    quote_currency=split_symbol[1]
    filename=base_currency+quote_currency+'_'+timeframe+'.csv'
    old_data=pd.read_csv('data/'+filename)
    print(old_data)
    max_timestamp=old_data['unix'].max()

    latest_price_data=price.get_price_data(symbol,interval,since=max_timestamp)

    new_data=latest_price_data[latest_price_data['unix']>=max_timestamp]
    old_data.drop(old_data['unix'].idxmax())

    new_file=old_data.append(new_data,ignore_index=True)

    print(new_file)






In [ ]:
def get_sma(data,window, close=True):
     #using daily for now
    timestamps=data['unix'][window-1:]
    if close:
        sma=data.rolling(window).mean()['close'].dropna()
    else:
        sma=data.rolling(window).mean()['open'].dropna()
    return pd.DataFrame({'unix': timestamps,'value':sma})

    # return pd.DataFrame({'unix': list(map(lambda x: x[0], sma)),'value':list(map(lambda x: x[1], sma))})

def get_ema(data,window, close=False):
    timestamps=data['unix'][window:]
    if close:
        ema=data.ewm(span=window,min_periods=window+1, adjust=False).mean()['close'].dropna()
    else:
        ema=data.ewm(span=window,min_periods=window+1, adjust=False).mean()['open'].dropna()
    return pd.DataFrame({'unix': timestamps,'value':ema})

def get_dema(data,window,close=False):
    ema=get_ema(data,window)
    ema=ema.rename(columns={'value':'open'})
    smoothed_ema=get_ema(ema,window)
    #making both vectors the same length
    start=np.min(smoothed_ema.index.values)
    ema=ema.loc[start:]
    timestamps=ema['unix'].values
    ema=ema['open'].values
    smoothed_ema=smoothed_ema['value'].values
    dema=2*ema-smoothed_ema
    return pd.DataFrame({'unix': timestamps,'value':dema})

def risk_indicator(fast,slow):
    min_timestamp=max(fast['unix'].min(),slow['unix'].min())

    trimmed_fast=fast.loc[fast['unix']>=min_timestamp]
    slow=slow.loc[slow['unix']>=min_timestamp]
    if len(trimmed_fast)>len(slow): 
        #different values, ie using a daily for fast and weekly for slow
        if (slow['unix'].max()<trimmed_fast['unix'].max()):
            print('true')
            #add another value to the slow moving avarage to facilitate interpolation
            slow=slow.append({'unix': trimmed_fast['unix'].max(), 'value':slow.iloc[-1]['value']},ignore_index=True)
        f=interp1d(slow['unix'],slow['value'])
        slow_interpolated=f(trimmed_fast['unix'])
        slow=pd.DataFrame({'unix':trimmed_fast['unix'],'value':slow_interpolated})

    if ('close' in fast.columns.values.tolist()):
        #using price
        risk_metric=np.divide(trimmed_fast['close'],slow['value'])
    else:
        #using moving average
        risk_metric=np.divide(trimmed_fast['value'],slow['value'])

    mean=np.mean(risk_metric)
    sigma=np.std(risk_metric)
    normalised=(risk_metric-mean)/sigma
    risk=norm.cdf(normalised)
    return pd.DataFrame({'unix':trimmed_fast['unix'],'value':risk})



In [ ]:
fast=get_sma(btc_price_data,10)
fast_ema=get_ema(btc_price_data,50)
slow=get_sma(weekly_candles,50)

In [ ]:
month_dictionary={
    'Jan':1,
    'Feb':2,
    'Mar':3,
    'Apr':4,
    'May':5,
    'Jun':6,
    'Jul':7,
    'Aug':8,
    'Sep':9,
    'Oct':10,
    'Nov':11,
    'Dec':12,
}

def parse_data(datestring):
    string=datestring.split('-')
    month=month_dictionary[string[0]]
    day=int(string[1])
    year=int(string[2])
    d=date(year,month,day)

    return int(time.mktime(d.timetuple()))
    


In [ ]:
def process_coincodex_csv(filename):

    raw_data=pd.read_csv('data/'+filename)
    vectorised_parsing=np.vectorize(parse_data)
    timestamps=vectorised_parsing(raw_data['Date'].values)
    df=raw_data

    df['unix']=timestamps
    df.columns = ['date','open','high','low','close','volume','market cap','unix']
    df.sort_values(by='unix', ignore_index=True, inplace=True)
    df.to_csv('data/'+filename+'_updated')

    return df

In [ ]:
def find_intercepts(fast,slow): #each line is a dataframe with time stamp and value, assuming both have same length
    #starting state
    min_timestamp=max(fast['unix'].min(),slow['unix'].min())

    fast=fast.loc[fast['unix']>=min_timestamp]
    slow=slow.loc[slow['unix']>=min_timestamp]

    if len(fast) != len(slow):
        f=interp1d(slow['unix'],slow['value'])
        slow_interpolated=f(fast['unix'])
        slow=pd.DataFrame({'unix':fast['unix'],'value':slow_interpolated})

    timestamps=fast['unix'].values
    line1=fast['value'].values
    line2=slow['value'].values

    line1_above_line2=line1[0]>line2[0] #state
    cross_above=[]
    cross_below=[]
    for i in range(len(line1)):
        if line1_above_line2 and line1[i]<line2[i]:
            cross_below.append(timestamps[i])
        elif not(line1_above_line2) and line1[i]>line2[i]:
            cross_above.append(timestamps[i])
        
        line1_above_line2 = line1[i]>line2[i]

    return {'cross_above': cross_above,'cross_below': cross_below }

In [5]:
eth=await get_stored_data('ETH/USD','1d')

In [6]:
def ma_channel(data, window):
    timestamps=data['unix']
    sma=data.rolling(window).mean()
    sma['unix']=timestamps
    sma.dropna(inplace=True)    

    return pd.DataFrame({'unix':sma['unix'],'high':sma['high'], 'low':sma['low'], 'value':sma['open']})



In [8]:
def backtest_ma_channel(data,period, slow_ma_gradient):
    print('Datapoints: ',len(data))
    equity=1

    channel=ma_channel(data,period)
    start=channel['unix'].min()
    trimmed_data=channel.loc[channel['unix']>=start]

    state='neutral'
    equity_record=[]
    longs=[]
    shorts=[]
    outcome=[]
    profit=[]
    for i in range(len(channel)):
        upper_bound=channel.iloc[i]['high']
        lower_bound=channel.iloc[i]['low']
        time=channel.iloc[i]['unix']
        
        day=max(list(filter(lambda x: x <= time, slow_ma_gradient.index.values)))
        gradient=slow_ma_gradient.loc[day]
        five_opens=data.loc[data['unix']<=time].tail(n=5)['open'].values # uses opens now as opens are confirmed
        current=five_opens[-1]
        uptrend=gradient>0
        
        if all(opens>upper_bound for opens in five_opens) and state != 'long' and uptrend.all():
            #remember to calculate profit if flipping from short
            if state=='short':
                outcome.append(current<entry)
                profit.append(1-(current/entry))
                equity=equity*(1+(1-(current/entry)))
                # equity=(entry*trade_amount-current*trade_amount)+equity
            state='long'
            entry=current
            longs.append(time)
        elif all(opens<lower_bound for opens in five_opens) and state != 'short' and not(uptrend.all()):
            if state=='long':
                outcome.append(current>entry)
                profit.append(current/entry -1)
                equity=equity*(1+(current/entry -1))
            entry=current
            # trade_amount=equity*entry
            state='short'
            shorts.append(time)
            #print(state+' from $'+str(current))
        if equity < 0:
            equity=0
            print('went broke')
            break
    
    return longs,shorts,np.array(outcome), np.array(profit), equity

def get_gradient(ma):
    
    return pd.Series(
        index=ma['unix'].values,
        data=np.gradient(ma['value'])
    )



# equity_curve=backtest_ma_channel(eth_hourly,20)[2]


In [9]:
eth=read_data('Bitstamp_ETHUSD_d.csv')
eth_hourly=read_data('Bitstamp_ETHUSD_1h.csv')
# eth=price.get_price_data('1d', symbol='ETH/USD')
# eth_hourly=price.get_price_data('1h', symbol='ETH/USD')

In [10]:

results=[]
for ema_period in range(1,8):
    print('ema=', ema_period)
    ema=get_ema(eth,ema_period, False)
    ema_gradient=get_gradient(ema)
    dema=get_dema(eth_hourly,ema_period,False)
    dema_gradient=get_gradient(dema)
    for ma_channel_period in range(24,30):
        print('ma=', ma_channel_period)
        longs, shorts, outcome, profit, final_equity=backtest_ma_channel(eth_hourly, ma_channel_period,ema_gradient)
        result=[ema_period,ma_channel_period,np.sum(outcome)/np.size(outcome),profit, final_equity]
        print('testing with dema now:')
        longs, shorts, outcome, profit, final_equity=backtest_ma_channel(eth_hourly, ma_channel_period,dema_gradient)
        result.extend([ema_period,np.sum(outcome)/np.size(outcome),profit, final_equity])
        results.append(result)
        

ema= 1
ma= 24
Datapoints:  27787
Datapoints:  27787


KeyboardInterrupt: 

In [ ]:
results_df=pd.DataFrame(np.array(results),columns=['ema','channel','success_rate','profit', 'final_equity', 'dema','success_rate','profit', 'final_equity'])
results_df

In [ ]:
filtered=results_df[results_df['final_equity']>30000]
profit_arrays=filtered['profit'].values
win_array=[]
loss_array=[]
max_profit=[]
max_drawdown=[]
number_of_trades=[]
for i in range(len(profit_arrays)):
    array=profit_arrays[i]
    win_filter=array>0
    wins=array[win_filter]
    loss_filter=array<0
    loss=array[loss_filter]
    win_array.append(wins)
    max_profit.append(max(wins))
    loss_array.append(loss)
    max_drawdown.append(min(loss))
    number_of_trades.append(len(array))
    assert len(wins)/(len(array))==filtered.iloc[i]['success_rate']
filtered['wins']=win_array
filtered['losses']=loss_array
filtered['max_profit']=max_profit
filtered['max_drawdown']=max_drawdown
filtered['number_of_trades']=number_of_trades
filtered

In [ ]:
best_result=filtered.iloc[0]
wins=pd.Series(best_result['wins'])
loss=pd.Series(best_result['losses'])
print(wins.describe(),loss.describe())

In [ ]:
ema=get_ema(eth,2, False)
gradient=get_gradient(ema)
channel=ma_channel(eth_hourly,26)
longs, shorts, outcome, profit, final_equity=backtest_ma_channel(eth_hourly, 26,gradient)
print('done')


In [ ]:
#testing on recent data
recent_eth_hourly=price.get_price_data('1h', symbol='ETH/USD')
recent_eth_daily=price.get_price_data('1d', symbol='ETH/USD')

results2=[]
# for ema_period in range(10,50):
#     print('ema=', ema_period)
ema=get_ema(recent_eth_daily,2, False)
gradient=get_gradient(ema)
channel=ma_channel(recent_eth_hourly,26)
#     for ma_channel_period in range(10,50):
#         print('ma=', ma_channel_period)
longs, shorts, outcome, profit, final_equity=backtest_ma_channel(recent_eth_hourly, 26,gradient)
#         if len(outcome)==0:
#             print('skipping')
#             break
#         print(np.sum(outcome)/len(outcome))
#         print(np.mean(profit))
#         print(final_equity)

# results2.append([ema_period,ma_channel_period,np.sum(outcome)/np.size(outcome),profit, final_equity])
# results_df2=pd.DataFrame(np.array(results),columns=['ema','channel','success_rate','profit', 'final_equity'])


In [ ]:
np.sum(outcome)/np.size(outcome),profit, final_equity

In [ ]:
print(np.sum(outcome)/len(outcome))
np.mean(profit)

In [ ]:
ethbtc=process_coincodex_csv('ETHBTC_d.csv')
risk=risk_indicator(ethbtc,get_sma(ethbtc,350))
fig1,ax1 = plt.subplots()
ax1.plot(risk['unix'],risk['value'])